# Solution to Task 4: State Preparation Circuit for 5 Qubits

### Universal State Preparation Circuit
I start with the universal state preparation circuit based on the work from [arXiv:1003.5760](https://arxiv.org/abs/1003.5760) based on a Schmidt decomposition structure. 

However, this apprach require two CNOT gates between non-connected qubits. To address this problem, I decompose these two CNOT gates into other CNOT that are hardware realizable using an equality in [arXiv:2009.13247](https://arxiv.org/pdf/2009.13247).

This method results in a circuit depth of 127 with 47 CNOT gates.

### Circuit Simplification
Since we focus on a specific target state, I simplify the circuit structure to achieve better circuit depth and reduce the number of CNOT gates and parameters. This is achieved by systematically removing gates while preserving the overall structure, ensuring that the fidelity of the state preparation remains at one.

### Transpilation with Qiskit
I further simplify the circuit by applying Qiskit transpilation, which employs simple rules for circuit simplification, as outlined in the [PennyLane tutorial](https://pennylane.ai/qml/demos/tutorial_circuit_compilation/).

## Results
The final circuit achieve the target state with:
- **Fidelity**: 1
- **Circuit Depth**: 38
- **Number of CNOT Gates**: 16


## References
1. [arXiv:1003.5760](https://arxiv.org/abs/1003.5760)
2. [arXiv:2009.13247](https://arxiv.org/pdf/2009.13247)
3. [PennyLane Tutorial](https://pennylane.ai/qml/demos/tutorial_circuit_compilation/)


# Universal State Preparation Circuit 

In [21]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA
from qiskit.transpiler import CouplingMap
from scipy.optimize import minimize

import numpy as np
from typing import List, Tuple
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import svd

In [22]:
def create_custom_statevector(size: int, state_values: List[int]) -> Statevector:
    """
    Create a custom state vector with specified basis states.

    Args:
        size (int): The number of qubits, determining the size of the state vector.
        state_values (List[int]): A list of indices representing the basis states to be included in the state vector.

    Returns:
        Statevector: A normalized state vector with specified amplitudes for the given basis states.
    """
    # Initialize a zero state vector of size 2^size
    state_vector = [0] * (2 ** size)
    
    # Set amplitudes for specified states to 0.5
    for s in state_values:
        state_vector[s] = 0.5
        
    # Convert to a complex numpy array for normalization
    vector = np.array(state_vector, dtype=complex)
    
    # Normalize the state vector
    norm = np.sqrt(np.sum(np.abs(vector)**2))
    vector = vector / norm
    
    # Print the binary representation of the states with their amplitudes
    print("Target state components:")
    for s in state_values:
        print(f"|{np.binary_repr(s, size)}⟩ with amplitude {0.5/norm:.4f}")
    
    return Statevector(vector)

def create_random_statevector(size: int) -> Statevector:
    """
    Create a random state vector of specified size with real amplitudes.

    Args:
        size (int): The number of qubits, determining the size of the state vector.

    Returns:
        Statevector: A normalized state vector with random amplitudes.
    """
    # Generate random amplitudes for the state vector
    state_vector = np.random.rand(2 ** size)
    
    # Normalize the state vector
    norm = np.linalg.norm(state_vector)
    state_vector = state_vector / norm
    
    # Print the binary representation of the states with their amplitudes
    print("Random state components:")
    for i in range(2 ** size):
        print(f"|{np.binary_repr(i, size)}⟩ with amplitude {state_vector[i]:.4f}")
    
    return Statevector(state_vector)

In [23]:
def universal_two_qubit(params):
    """
    Create a universal two-qubit gate using specified parameters.

    Args:
        params (list): A list of 15 parameters for the gate operations.

    Returns:
        QuantumCircuit: A quantum circuit implementing the universal two-qubit gate.
    """
    # Create circuit for the universal gate
    qc = QuantumCircuit(2)
    
    # First local unitary on qubit 0
    qc.h(0)
    qc.rz(params[0], 0)
    qc.h(0)
    qc.rz(params[1], 0)
    
    # First local unitary on qubit 1
    qc.h(1)
    qc.rz(params[2], 1)
    qc.h(1)
    qc.rz(params[3], 1)
    
    # Non-local part with 3 CX gates
    qc.cx(0, 1)

    qc.h(0)
    qc.rz(params[4], 0)
    qc.rz(params[5], 1)
    qc.h(0)
    qc.rz(params[6], 0)
    
    qc.cx(1, 0)
    
    qc.rz(params[7], 1)
    qc.rz(params[8], 0)
    qc.h(0)
    qc.rz(params[9], 0)
    qc.h(0)
    
    qc.cx(0, 1)
    
    # Second local unitary on qubit 0
    qc.rz(params[10], 0)
    qc.h(0)
    qc.rz(params[11], 0)
    
    # Second local unitary on qubit 1
    qc.h(1)
    qc.rz(params[12], 1)
    qc.h(1)
    qc.rz(params[13], 1)
    
    # Final phase
    qc.h(0)
    
    return qc


def multiplexed_r_gate(qc, qr, start_idx, n_qubits, params, rot):
    """
    Recursive helper function for multiplexed R gate implementation.
    
    Args:
        qc (QuantumCircuit): Existing quantum circuit.
        qr (QuantumRegister): Quantum register.
        start_idx (int): Starting index for this subset of qubits.
        n_qubits (int): Number of qubits to use from start_idx.
        params (list): Rotation parameters.
        rot (str): Type of rotation ('rx' or 'ry').
    """
    # Base case for 2 qubits
    if n_qubits == 2:
        if rot == 'rx':
            qc.rx(params[0], qr[-1])
            qc.cx(qr[-2], qr[-1])
            qc.rx(params[1], qr[-1])
            qc.cx(qr[-2], qr[-1])
        elif rot == 'ry':
            qc.h(qr[-1])
            qc.rx(params[0], qr[-1])
            qc.h(qr[-1])
            qc.cx(qr[-2], qr[-1])
            qc.h(qr[-1])
            qc.rx(params[1], qr[-1])
            qc.h(qr[-1])
            qc.cx(qr[-2], qr[-1])
    else:
        # Split parameters into two groups
        n_params = len(params)
        params1 = params[:n_params // 2]
        params2 = params[n_params // 2:]
        
        # Recursively apply to n-1 qubits
        multiplexed_r_gate(qc, qr, start_idx + 1, n_qubits - 1, params1, rot)
        qc.cx(qr[start_idx], qr[-1])
        multiplexed_r_gate(qc, qr, start_idx + 1, n_qubits - 1, params2, rot)
        qc.cx(qr[start_idx], qr[-1])


def universal_three_qubit(params):
    """
    Create a universal three-qubit gate using specified parameters.

    Args:
        params (list): A list of parameters for the gate operations.

    Returns:
        QuantumCircuit: A quantum circuit implementing the universal three-qubit gate.
    """
    # Create circuit for the universal gate
    qr = QuantumRegister(3)
    qc = QuantumCircuit(qr)

    qc = qc.compose(universal_two_qubit(params[:14]), [0, 1])
    multiplexed_r_gate(qc, qr, 0, 3, params[14:18], 'rx')
    qc = qc.compose(universal_two_qubit(params[18:32]), [0, 1])
    multiplexed_r_gate(qc, qr, 0, 3, params[32:36], 'ry')

    qc = qc.compose(universal_two_qubit(params[36:50]), [0, 1])
    multiplexed_r_gate(qc, qr, 0, 3, params[50:54], 'rx')

    qc = qc.compose(universal_two_qubit(params[54:68]), [0, 1])

    return qc


def state_prep_circuit(params):
    """
    Creates a parameterized quantum circuit for five-qubit state preparation.

    Args:
        params (list): List of parameters for the circuit.

    Returns:
        QuantumCircuit: A quantum circuit for state preparation.
    """
    qr = QuantumRegister(5, 'q')
    circuit = QuantumCircuit(qr)
    
    param_idx = 0
    
    # Phase 1:
    circuit.h(qr[0])
    circuit.rz(params[param_idx], qr[0])
    circuit.h(qr[0])
    circuit.rz(params[param_idx + 1], qr[0])

    circuit.cx(qr[0], qr[1])

    circuit.h(qr[1])
    circuit.rz(params[param_idx + 2], qr[1])
    circuit.h(qr[1])
    circuit.rz(params[param_idx + 3], qr[1])
    
    circuit.h(qr[0])
    circuit.rz(params[param_idx + 4], qr[0])
    circuit.h(qr[0])
    circuit.rz(params[param_idx + 5], qr[0])

    circuit.cx(qr[0], qr[1])

    param_idx += 6
    
    # Phase 2:  
    circuit.cx(qr[0], qr[4])
    circuit.cx(qr[4], qr[2])
    circuit.cx(qr[0], qr[4])  # Undo the intermediate CNOT
    circuit.cx(qr[4], qr[2])
    
    circuit.cx(qr[1], qr[4])
    circuit.cx(qr[4], qr[3])
    circuit.cx(qr[1], qr[4])
    circuit.cx(qr[4], qr[3])

    # Phase 3:
    circuit = circuit.compose(universal_two_qubit(params[param_idx:param_idx + 14]), [0, 1])
    param_idx += 14
    
    # Phase 4:
    circuit = circuit.compose(universal_three_qubit(params[param_idx:param_idx + 68]), [2, 3, 4])
    
    return circuit


def cost_function(params_values, circuit, params, target_state):
    """
    Calculate the cost (1 - fidelity) between the prepared state and target state.

    Args:
        params_values (np.array): Values for the rotation parameters.
        circuit (QuantumCircuit): Parameterized circuit.
        params (list): List of Parameter objects.
        target_state (Statevector): Target quantum state.
        
    Returns:
        float: Cost value (1 - fidelity).
    """
    # Bind the parameters
    bound_circuit = circuit.assign_parameters(dict(zip(params, params_values)))
    
    # Get the state vector from the circuit
    prepared_state = Statevector.from_instruction(bound_circuit)
    
    # Calculate fidelity
    fidelity = state_fidelity(prepared_state, target_state)
    
    return 1 - fidelity


def optimize_circuit(target_state_vector, circuit, params, n_trials=5):
    """
    Creates and optimizes a circuit to prepare the target state.

    Args:
        target_state_vector (np.array): Target state vector (32-dimensional for 5 qubits).
        n_trials (int): Number of optimization attempts with random starting points.
        
    Returns:
        tuple: (optimized circuit, final fidelity).
    """
    # Convert target state vector to Statevector object
    target_state = Statevector(target_state_vector)
    
    best_fidelity = -1
    best_result = None
    
    # Try multiple random starting points
    for trial in range(n_trials):
        # Initial parameter values
        initial_params = np.random.rand(len(params)) * 2 * np.pi
        
        # Define bounds for parameters (all rotations are between 0 and 2π)
        bounds = [(0, 2 * np.pi) for _ in params]
        
        # Optimize parameters
        result = minimize(
            cost_function,
            initial_params,
            args=(circuit, params, target_state),
            method='SLSQP',
            bounds=bounds,
            options={'maxiter': 1000}
        )
        
        fidelity = 1 - result.fun
        
        if fidelity > best_fidelity:
            best_fidelity = fidelity
            best_result = result
        
        print(f"Trial {trial + 1}: Fidelity = {fidelity:.6f}")
    
    # Create the optimized circuit using best parameters
    optimized_circuit = circuit.assign_parameters(
        dict(zip(params, best_result.x))
    )
    
    return optimized_circuit, best_fidelity

In [28]:
# Define the size of the quantum state and the target state values
size = 5
state_values = [22, 17, 27, 12]

# Create the target state using a custom state vector
target_state = create_custom_statevector(size, state_values)

print("Starting optimization...")

# Define parameters for the circuit (88 parameters for the state preparation)
params = [Parameter(f'θ_{i}') for i in range(88)]

# Create and optimize the circuit with multiple trials
optimized_circuit, fidelity = optimize_circuit(target_state, state_prep_circuit(params), params, n_trials=5)

print(f"\nOptimization complete!")
print(f"Final fidelity: {fidelity:.6f}")
print(f"Circuit depth: {optimized_circuit.depth()}")
print(f"Number of CNOTs: {sum(1 for inst in optimized_circuit.data if inst.operation.name == 'cx')}")

# Draw the optimized circuit
print("\nOptimized Circuit:")
print(optimized_circuit.draw())


Target state components:
|10110⟩ with amplitude 0.5000
|10001⟩ with amplitude 0.5000
|11011⟩ with amplitude 0.5000
|01100⟩ with amplitude 0.5000
Starting optimization...
Trial 1: Fidelity = 1.000000
Trial 2: Fidelity = 1.000000
Trial 3: Fidelity = 1.000000
Trial 4: Fidelity = 1.000000
Trial 5: Fidelity = 0.853553

Optimization complete!
Final fidelity: 1.000000
Circuit depth: 127
Number of CNOTs: 43

Optimized Circuit:
     ┌───┐┌────────────┐┌───┐┌────────────┐     ┌───┐┌────────────┐┌───┐»
q_0: ┤ H ├┤ Rz(4.7128) ├┤ H ├┤ Rz(1.3284) ├──■──┤ H ├┤ Rz(5.6595) ├┤ H ├»
     └───┘└────────────┘└───┘└────────────┘┌─┴─┐├───┤├────────────┤├───┤»
q_1: ──────────────────────────────────────┤ X ├┤ H ├┤ Rz(4.1029) ├┤ H ├»
                                           └───┘└───┘└────────────┘└───┘»
q_2: ───────────────────────────────────────────────────────────────────»
                                                                        »
q_3: ──────────────────────────────────────────────────────

In [30]:
# Transpile the optimized circuit to a specific basis and optimization level
transpiled_circuit = transpile(optimized_circuit, basis_gates=['x', 'cx', 'h', 'rz'], optimization_level=2)

# Print the depth of the transpiled circuit
print(f"Circuit depth: {transpiled_circuit.depth()}")

# Count and print the number of CNOT gates in the transpiled circuit
print(f"Number of CNOTs: {sum(1 for inst in transpiled_circuit.data if inst.operation.name == 'cx')}")

# View the transpiled circuit
print(transpiled_circuit)


Circuit depth: 100
Number of CNOTs: 25
     ┌───┐┌────────────┐┌───┐┌────────────┐     ┌───┐┌────────────┐┌───┐»
q_0: ┤ H ├┤ Rz(4.7128) ├┤ H ├┤ Rz(1.3284) ├──■──┤ H ├┤ Rz(5.6595) ├┤ H ├»
     └───┘└────────────┘└───┘└────────────┘┌─┴─┐├───┤├────────────┤├───┤»
q_1: ──────────────────────────────────────┤ X ├┤ H ├┤ Rz(4.1029) ├┤ H ├»
                                           └───┘└───┘└────────────┘└───┘»
q_2: ───────────────────────────────────────────────────────────────────»
                                                                        »
q_3: ───────────────────────────────────────────────────────────────────»
                                                                        »
q_4: ───────────────────────────────────────────────────────────────────»
                                                                        »
«     ┌────────────┐                    ┌───┐┌────────────┐┌───┐┌────────────┐»
«q_0: ┤ Rz(1.7809) ├──■────■─────────■──┤ H ├┤ Rz(5.2469) ├┤ H ├┤ R

# Circuit Simplification

In [31]:
def universal_two_qubit_simplified(params):
    """
    Create a simplified universal two-qubit gate using specified parameters.

    Args:
        params (list): A list of 4 parameters for the gate operations.

    Returns:
        QuantumCircuit: A quantum circuit implementing the simplified universal two-qubit gate.
    """
    # Create circuit for the universal gate
    qc = QuantumCircuit(2)
    
    # First local unitary on qubit 0
    qc.h(0)
    qc.rz(params[0], 0)
    
    # First local unitary on qubit 1
    qc.h(1)
    qc.rz(params[1], 1)
    
    # Non-local part with CX gate
    qc.cx(0, 1)

    # Apply additional rotations
    qc.h(0)
    qc.rz(params[2], 0)
    qc.rz(params[3], 1)

    return qc

def multiplexed_r_gate_simplified(qc, qr, start_idx, n_qubits, params, rot):
    """
    Recursive helper function for multiplexed R gate implementation.
    
    Args:
        qc (QuantumCircuit): Existing quantum circuit.
        qr (QuantumRegister): Quantum register.
        start_idx (int): Starting index for this subset of qubits.
        n_qubits (int): Number of qubits to use from start_idx.
        params (list): Rotation parameters.
        rot (str): Type of rotation ('rx' or 'ry').
    """
    # Base case for 2 qubits
    if n_qubits == 2:
        if rot == 'rx':
            qc.rx(params[0], qr[-1])
            qc.cx(qr[-2], qr[-1])
            qc.rx(params[1], qr[-1])
            qc.cx(qr[-2], qr[-1])
        elif rot == 'ry':
            qc.h(qr[-1])
            qc.rx(params[0], qr[-1])
            qc.h(qr[-1])
            qc.cx(qr[-2], qr[-1])
            qc.h(qr[-1])
            qc.rx(params[1], qr[-1])
            qc.h(qr[-1])
            qc.cx(qr[-2], qr[-1])
    else:
        # Split parameters into two groups
        n_params = len(params)
        params1 = params[:n_params // 2]
        params2 = params[n_params // 2:]
        
        # Recursively apply to n-1 qubits
        multiplexed_r_gate_simplified(qc, qr, start_idx + 1, n_qubits - 1, params1, rot)
        qc.cx(qr[start_idx], qr[-1])
        multiplexed_r_gate_simplified(qc, qr, start_idx + 1, n_qubits - 1, params2, rot)
        qc.cx(qr[start_idx], qr[-1])

def universal_three_qubit_simplified(params):
    """
    Create a simplified universal three-qubit gate using specified parameters.

    Args:
        params (list): A list of parameters for the gate operations.

    Returns:
        QuantumCircuit: A quantum circuit implementing the simplified universal three-qubit gate.
    """
    # Create circuit for the universal gate
    qr = QuantumRegister(3)
    qc = QuantumCircuit(qr)

    # Compose the simplified two-qubit gates and multiplexed R gates
    qc = qc.compose(universal_two_qubit_simplified(params[:4]), [0, 1])
    multiplexed_r_gate_simplified(qc, qr, 0, 3, params[4:8], 'rx')
    qc = qc.compose(universal_two_qubit_simplified(params[8:12]), [0, 1])
    multiplexed_r_gate_simplified(qc, qr, 0, 3, params[12:16], 'ry')

    qc = qc.compose(universal_two_qubit_simplified(params[16:20]), [0, 1])
    multiplexed_r_gate_simplified(qc, qr, 0, 3, params[20:24], 'rx')

    qc = qc.compose(universal_two_qubit_simplified(params[24:28]), [0, 1])

    return qc


In [32]:
def state_prep_circuit_simplified(params):
    """
    Creates a parameterized quantum circuit for five-qubit state preparation
    using only X, H, RZ, and CX gates with connectivity constraints:
    0-1, 0-4, 1-4, 2-3, 2-4, 3-4.

    Args:
        params (list): List of parameters for the circuit.

    Returns:
        QuantumCircuit: A quantum circuit for state preparation.
    """
    qr = QuantumRegister(5, 'q')
    circuit = QuantumCircuit(qr)
    
    # Initialize parameter index for tracking parameter usage
    param_idx = 0
    
    # Phase 1: Prepare the first qubit and entangle with the second
    circuit.h(qr[0])  # Apply Hadamard to qubit 0
    circuit.rz(params[param_idx], qr[0])  # Apply RZ rotation on qubit 0

    circuit.cx(qr[0], qr[1])  # CNOT from qubit 0 to qubit 1

    circuit.h(qr[1])  # Apply Hadamard to qubit 1
    circuit.rz(params[param_idx + 1], qr[1])  # Apply RZ rotation on qubit 1
    
    param_idx += 2  # Update parameter index
    
    # Phase 2: Create additional entanglements
    circuit.cx(qr[0], qr[4])  # CNOT from qubit 0 to qubit 4
    circuit.cx(qr[4], qr[2])  # CNOT from qubit 4 to qubit 2

    circuit.cx(qr[1], qr[4])  # CNOT from qubit 1 to qubit 4
    circuit.cx(qr[4], qr[3])  # CNOT from qubit 4 to qubit 3

    # Phase 3: Add a universal two-qubit gate on the first two qubits
    circuit = circuit.compose(universal_two_qubit_simplified(params[param_idx:param_idx + 4]), [0, 1])
    param_idx += 4  # Update parameter index
    
    # Phase 4: Add a universal three-qubit gate
    circuit = circuit.compose(universal_three_qubit_simplified(params[param_idx:param_idx + 28]), [2, 3, 4])
    
    return circuit


In [33]:
# Define the size of the quantum state and the target state values
size = 5
state_values = [22, 17, 27, 12]

# Create the target state using a custom state vector
target_state = create_custom_statevector(size, state_values)

print("Starting optimization...")

# Define parameters for the circuit (34 parameters for the state preparation)
params = [Parameter(f'θ_{i}') for i in range(34)]

# Create and optimize the circuit with multiple trials
optimized_circuit, fidelity = optimize_circuit(target_state, state_prep_circuit_simplified(params), params, n_trials=3)

print(f"\nOptimization complete!")
print(f"Final fidelity: {fidelity:.6f}")
print(f"Circuit depth: {optimized_circuit.depth()}")
print(f"Number of CNOTs: {sum(1 for inst in optimized_circuit.data if inst.operation.name == 'cx')}")

# Draw the optimized circuit
print("\nOptimized Circuit:")
print(optimized_circuit.draw())

Target state components:
|10110⟩ with amplitude 0.5000
|10001⟩ with amplitude 0.5000
|11011⟩ with amplitude 0.5000
|01100⟩ with amplitude 0.5000
Starting optimization...
Trial 1: Fidelity = 0.999999
Trial 2: Fidelity = 0.999999
Trial 3: Fidelity = 0.853553

Optimization complete!
Final fidelity: 0.999999
Circuit depth: 57
Number of CNOTs: 28

Optimized Circuit:
     ┌───┐┌────────────┐                   ┌───┐     ┌────────────┐     »
q_0: ┤ H ├┤ Rz(3.0859) ├──■─────────■──────┤ H ├─────┤ Rz(1.5716) ├─────»
     └───┘└────────────┘┌─┴─┐┌───┐  │  ┌───┴───┴────┐└────────────┘┌───┐»
q_1: ───────────────────┤ X ├┤ H ├──┼──┤ Rz(4.7677) ├──────■───────┤ H ├»
                        └───┘└───┘  │  └───┬───┬────┘      │       ├───┤»
q_2: ───────────────────────────────┼──────┤ X ├───────────┼───────┤ H ├»
                                    │      └─┬─┘           │       ├───┤»
q_3: ───────────────────────────────┼────────┼─────────────┼───────┤ X ├»
                                  ┌─┴─┐     

# Transpilation with Qiskit

In [34]:
# Transpile the optimized circuit to a specific basis and optimization level
transpiled_circuit = transpile(optimized_circuit, basis_gates=['x', 'cx', 'h', 'rz'], optimization_level=2)

# Print the depth of the transpiled circuit
print(f"Circuit depth: {transpiled_circuit.depth()}")

# Count and print the number of CNOT gates in the transpiled circuit
print(f"Number of CNOTs: {sum(1 for inst in transpiled_circuit.data if inst.operation.name == 'cx')}")

# View the transpiled circuit
print(transpiled_circuit)

Circuit depth: 38
Number of CNOTs: 16
         ┌───┐    ┌────────────┐                         ┌───┐     »
q_0: ────┤ H ├────┤ Rz(3.0859) ├─────■────────────■──────┤ H ├─────»
         └───┘    └────────────┘   ┌─┴─┐   ┌───┐  │  ┌───┴───┴────┐»
q_1: ──────────────────────────────┤ X ├───┤ H ├──┼──┤ Rz(4.7677) ├»
                                   └───┘   └───┘  │  └────────────┘»
q_2: ─────────────────────────────────────────────┼────────────────»
     ┌───────────┐    ┌───┐     ┌─────────┐┌───┐  │  ┌────────────┐»
q_3: ┤ Rz(-3π/2) ├────┤ H ├─────┤ Rz(π/2) ├┤ H ├──┼──┤ Rz(9.4253) ├»
     └───────────┘    └───┘     └─────────┘└───┘┌─┴─┐└────────────┘»
q_4: ───────────────────────────────────────────┤ X ├──────────────»
                                                └───┘              »
«     ┌────────────┐                                                ┌───┐     »
«q_0: ┤ Rz(1.5716) ├──────────────────────────────────────■─────────┤ H ├─────»
«     └────────────┘         ┌───┐     ┌───